# Workflow
This section shows how to generate consistent subcellular landmarks from an example 4i dataset.

CAMPA contains a high-level API together with command line scripts that can be easily used to create datasets, train models, and extract features.
Settings for the different stages of the workflow are communicated via parameter files. 
These are python files usually containing a dictionary of settings that are used by the individual steps.
You can find a complete set of example parameter files in [notebooks/params](params).

In the following we will use these parameter files to use the cVAE framework to generate CSLs for unperturbed and meayamycin perturbed cells.

## Setup & download data
Ensure that you have set up `config.ini` as described in the [installation instructions](../installation.rst) and have an entry for `TestData` ( `TestData = .../notebooks/params/TestData_constants.py`) in the `[data]` section.
The `[data]` section contains python files with configuration parameters for loading specific data. This data will be identified in CAMPA using the name set in the `[data]` section. 
E.g., for the test data, it will be identified by `TestData` and point to the config in [`notebooks/params/TestData_constants.py`](params/TestData_constants.py). 

The constants file is specific per dataset but has to contain the following variables:
- `DATA_DIR`: path to the folder containing the data
- `DATASET_DIR`: path to the folder where training/testing datasets derived from this data should be stored
- `OBJ_ID`: name of column in metadata.csv that contains a unique object identifier
- `CHANNELS_METADATA`: name of csv file containing channels metadata (relative to `DATA_DIR`). Is expected to contain channel names in column "name".
- `CONDITIONS`: dictionary of conditions to be used for cVAE models. Keys are column names in `metadata.csv`, and values are all possible values for this condition. This will be used to convert conditions to one-hot encoded vector

For using a different dataset, simply create a new constants file with dataset specific settings, and add a new entry to the `[data]` section in your `config.ini` file. 

To follow along with this tutorial, you need to download the example dataset.

TODO download commands

The example data is now stored in your `notebooks/example_data` folder.

The data is represented as an `MPPData` object. For more information on this class and the data representation on disk see [the tutorial on MPPData](mpp_data.ipynb).

## Create NNDataset
`NNDatasets` are created using a data params file specifying the data, normalisation, and subsampling for generating a NNDataset. 
Internally, an `NNDataset` represents train, val and test splits as `MPPData` objects. For more infomation on `NNDataset`, see [the tutorial on creating and working with a dataset using NNDataset](nn_dataset.ipynb)

The NNDataset can easily be created with the cli (assuming $CAMPA_DIR the location of campa):
```
campa create_dataset $CAMPA_DIR/notebooks/params/example_data_params.py
```

## Train cVAE
For training and evaluation, an `experiment_params.py` file is needed.
This file defines several experiments. For more information on the training and evaluation process, see [the tutorial on training cVAE models](train.ipynb)

To train, evaluate and cluster models, run (assuming $CAMPA_DIR the location of campa):
```
campa train all --config $CAMPA_DIR/notebooks/params/example_experiment_params.py
```
The models will be saved in `EXPERIMENT_DIR/test`, with the `EXPERIMENT_DIR` being the custom experiment path set up in `campa.ini`.


## Cluster cVAE latent representation
After training a cVAE model, we can use its latent representation to generate consistent subcellular landmarks.
This is done in three steps. For more information, see [the tutorial on clustering](cluster.ipynb).

TODO describe and link to detailled notebook

### Cluster a subset of the data
First, the data is subsampled and clustered. This is done, because it is not feasible to cluster >10 million datapoints.
If you have a small enough dataset, you might want to skip the subsampling step and cluster all data directly.
The following subsamples the data to 10%, and clusters it.
```
campa cluster test/CondVAE_pert-CC create --subsample --frac 0.1 --save-dir aggregated/sub-0.1 --cluster
```

Optionally, after this step a manual reclustering or annotation of clusters can be done. See [the tutorial](cluster.ipynb) for more details.

### Predict latent representation on entire dataset
To project the clustering to the entire dataset, the model needs to be used to predict the latent representation on all data. 
It is recommended to run this step in a script, as this might take a while for large datasets.
```
campa cluster test/CondVAE_pert-CC prepare-full --save-dir aggregated/full_data
```

### Project cluster assignments to entire dataset
Finally, the clustering can be project the to entire dataset:
```
campa cluster test/CondVAE_pert-CC project aggregated/sub-0.1 --save-dir aggregated/full_data --cluster-name clustering_res0.2
```


## Extract molecular and spatial features
After creating and annotating CSLs, features can be extracted from each cell to quantitatively compare molecular intensity differences and spatial relocalisation of proteins in different conditions. For more information on the features and how to test for differences and visualise results, have a look at [the feature extraction tutorial](extract_features.ipynb)
```
campa extract_features $CAMPA_DIR/notebooks/params/example_features_params.py
```
